In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa
import librosa.display
import noisereduce as nr 
from tqdm import tqdm



In [6]:
metadata_df = pd.read_csv('../Test_1.csv')
print(metadata_df.head())

              id     audio_filepath  duration
0  id_u5iqtgjzhx  id_u5iqtgjzhx.wav   2.35800
1  id_l7ebzcfk5e  id_l7ebzcfk5e.wav   3.22100
2  id_jbzci8uepl  id_jbzci8uepl.wav   2.94898
3  id_jzil0fw5vs  id_jzil0fw5vs.wav   4.33100
4  id_o7mrvf5wj7  id_o7mrvf5wj7.wav   1.85600


In [7]:
root_path = "../TechCabal Ewe Audio Files/"
audio_path = root_path + metadata_df.iloc[0, 1]
def denoise(sig, sr, threshold=2.0, is_stationary=True):
    return nr.reduce_noise(y=sig, sr=sr, n_std_thresh_stationary=threshold, stationary=is_stationary)


In [9]:
def preprocess_audio(file_path, max_length, max_duration):
    sig ,sr = librosa.load(file_path, duration=max_duration)
    sig = denoise(sig, sr)
    mfcc = librosa.feature.mfcc(y=sig, sr=sr, n_mfcc=40)
    if mfcc.shape[1] > max_length:
        mfcc = mfcc[:,:max_length]
    else:
        pad_width = max_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0), (0, pad_width)), mode='constant')
    return mfcc

extracted_features = []
target_mfcc_length = 115
max_audio_duration = metadata_df['duration'].max()

for idx in tqdm(range(len(metadata_df))):
    audio_path = root_path + metadata_df.loc[idx, 'audio_filepath']
    features = preprocess_audio(audio_path, target_mfcc_length, max_audio_duration)
    extracted_features.append([features])

100%|██████████| 2946/2946 [01:42<00:00, 28.71it/s]


In [10]:
extracted_features_df = pd.DataFrame(extracted_features, columns=['features']
                                     )
extracted_features_df.head()


,features
0,"[[-806.3182, -787.3213, -772.769, -761.94727, ..."
1,"[[-658.06433, -658.06433, -658.06433, -658.064..."
2,"[[-775.6121, -751.0812, -736.7386, -723.77893,..."
3,"[[-597.01044, -597.01044, -597.01044, -564.812..."
4,"[[-746.0511, -746.0511, -746.0511, -746.0511, ..."


In [11]:
extracted_features_df.loc[0, 'features']
extracted_features_df.loc[0, 'features'].shape

(40, 115)

In [12]:

features = extracted_features_df['features'].tolist()

In [13]:
print(len(features))


2946


In [14]:
# save features and labels separately
np.save('val_features.npy', features)

In [41]:
#load features and labels
features = np.load('features.npy')
labels = np.load('labels.npy')

print(features.shape)
print(labels.shape)

(5334, 40, 115)
(5334,)
